# Long text summarization using LCEL chains on Langchain with Bedrock APIs

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Overview
When we work with large documents, we can face some challenges as the input text might not fit into the model context length, or the model hallucinates with large documents, or, out of memory errors, etc.

To solve those problems, we are going to show a solution that is based on the concept of chunking and chaining prompts. This solution is leveraging [LangChain](https://python.langchain.com/docs/get_started/introduction.html) which is a popular framework for developing applications powered by language models.

In this architecture:

1. A large document (or a giant file appending small ones) is loaded
1. Langchain utility is used to split it into multiple smaller chunks (chunking)
1. First chunk is sent to the model; Model returns the corresponding summary
1. Langchain gets next chunk and appends it to the returned summary and sends the combined text as a new request to the model; the process repeats until all chunks are processed
1. In the end, you have final summary based on entire content

### Use case
This approach can be used to summarize call transcripts, meetings transcripts, books, articles, blog posts, and other relevant content.

### Imports

In [2]:
import os
import sys
from langchain_aws import ChatBedrockConverse
from IPython.core.display import HTML
from IPython.display import display_markdown, Markdown
import boto3

HTML("<script>Jupyter.notebook.kernel.restart()</script>")

module_path = ".."
sys.path.append(os.path.abspath(module_path))

boto3_bedrock = boto3.client('bedrock-runtime')

textgen_llm = ChatBedrockConverse(
    model_id="amazon.nova-lite-v1:0",
    client=boto3_bedrock,
    max_tokens=None,
    temperature=0.5
)


### Load shareholder letter

We will be following a process similar to lab 02 in this summarization section. First, let us load the 2022 Amazon shareholder letter

In [3]:
shareholder_letter = "./letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()

In [4]:
len(letter.split(' '))

5084

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [6]:
num_docs = len(docs)

num_tokens_first_doc = textgen_llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

/Users/claumazz/Projects/amazon-bedrock-workshop/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now we have 10 documents and the first one has 439 tokens


In [7]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import XMLOutputParser
from langchain.schema.output_parser import StrOutputParser


xml_parser = XMLOutputParser(tags=['insight'])
str_parser = StrOutputParser()

prompt = PromptTemplate(
    template="""
    
    Human:
    {instructions} : \"{document}\"
    Format help: {format_instructions}.
    Assistant:""",
    input_variables=["instructions","document"],
    partial_variables={"format_instructions": xml_parser.get_format_instructions()},
)

insight_chain = prompt | textgen_llm | StrOutputParser()

In [8]:
print(f"Number of Documents {len(docs)}")

Number of Documents 10


# Option 1. Manually process insights, then summarize

In [9]:
%%time
insights=[]
for i in range(len(docs)):
    insights.append(
        insight_chain.invoke({
        "instructions":"Provide Key insights from the following text",
        "document": {docs[i].page_content}
    }))

CPU times: user 68.3 ms, sys: 6.68 ms, total: 75 ms
Wall time: 30.1 s


In [10]:
str_parser = StrOutputParser()

prompt = PromptTemplate(
    template="""
    
    Human:
    {instructions} : \"{document}\"
    Assistant:""",
    input_variables=["instructions","document"]
)

summary_chain = prompt | textgen_llm | StrOutputParser()

In [11]:
%%time
display_markdown(Markdown(summary_chain.invoke({
        "instructions":"You will be provided with multiple sets of insights. Compile and summarize these insights and provide key takeaways in one concise paragraph. Do not use the original xml tags. Just provide a paragraph with your compiled insights.",
        "document": {'\n'.join(insights)}
    })))

Amazon has demonstrated remarkable resilience and adaptability, growing demand and innovating across its largest businesses despite facing tough macroeconomic conditions and internal challenges. Over the past 25 years, the company has evolved from a small books retailer to a global marketplace with a significant third-party seller ecosystem, and has expanded into cloud services with AWS and innovative products like Kindle and Alexa. Amazon has consistently adapted to changes, such as during the dot-com crash and the 2008-2009 recession, by making strategic investments and cost adjustments. The company has also ventured into new market segments, including grocery and business procurement, and has launched initiatives like Buy with Prime and Amazon Pharmacy to enhance customer experiences. Amazon is heavily investing in Large Language Models (LLMs) and Generative AI, aiming to innovate across its business for decades to come. With a strong focus on customer focus, relentless invention, and leading customer experiences, Amazon remains optimistic about its future growth as it continues to address market opportunities in retail and IT spending.

CPU times: user 6.36 ms, sys: 1.91 ms, total: 8.27 ms
Wall time: 2.03 s


Map reduce

# Option 2. Use Map reduce pattern on Langchain

In [14]:
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=textgen_llm, chain_type="map_reduce", verbose=False)

In [26]:
%%time
display_markdown(Markdown(summary_chain.invoke(docs)['input_documents'][0].page_content))

As I sit down to write my second annual shareholder letter as CEO, I find myself optimistic and energized by what lies ahead for Amazon. Despite 2022 being one of the harder macroeconomic years in recent memory, and with some of our own operating challenges to boot, we still found a way to grow demand (on top of the unprecedented growth we experienced in the first half of the pandemic). We innovated in our largest businesses to meaningfully improve customer experience short and long term. And, we made important adjustments in our investment decisions and the way in which we’ll invent moving forward, while still preserving the long-term investments that we believe can change the future of Amazon for customers, shareholders, and employees.

While there were an unusual number of simultaneous challenges this past year, the reality is that if you operate in large, dynamic, global market segments with many capable and well-funded competitors (the conditions in which Amazon operates all of its businesses), conditions rarely stay stagnant for long.

In the 25 years I’ve been at Amazon, there has been constant change, much of which we’ve initiated ourselves. When I joined Amazon in 1997, we had booked $15M in revenue in 1996, were a books-only retailer, did not have a third-party marketplace, and only shipped to addresses in the US. Today, Amazon sells nearly every physical and digital retail item you can imagine, with a vibrant third-party seller ecosystem that accounts for 60% of our unit sales, and reaches customers in virtually every country around the world. Similarly, building a business around a set of technology infrastructure services in the cloud was not obvious in 2003 when we started pursuing AWS, and still wasn’t when we launched our first services in 2006. Having virtually every book at your fingertips in 60 seconds, and then being able to store and retrieve them on a lightweight digital reader was not “a thing” yet when we launched Kindle in 2007, nor was a voice-driven personal assistant like Alexa (launched in 2014) that you could use to access entertainment, control your smart home, shop, and retrieve all sorts of information.

CPU times: user 52.8 ms, sys: 13 ms, total: 65.8 ms
Wall time: 16.1 s
